In [ ]:
!pip install --quiet transformers seaborn datasets evaluate rouge_score nltk numpy spacy lexrank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:0

In [ ]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/after_match_pairs_data.csv" /content/after_match_pairs_data.csv

In [ ]:
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

from lexrank import LexRank
from spacy.lang.ru.stop_words import STOP_WORDS

import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from datasets import Dataset
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
import evaluate

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('/content/after_match_pairs_data.csv')
data

,match_id,news_id,name,team1,team2,content,news_name,news_body,news_name_body
0,455452,141499717,Чемпионат Европы,Дания,Германия,19 минута. гол! Не Гомесом единым! Подольски п...,Ларс Бендер забил первый гол за сборную Германии,Полузащитник «Байера» и сборной Германии Ларс ...,Ларс Бендер забил первый гол за сборную Герман...
1,481837,143405274,Россия. Премьер-лига,Кубань,Спартак,15 минута. гол! Макгиди прибежал в штрафную на...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...
2,1089413,1037500549,Кубок Гагарина,СКА,Локомотив,15 минута. гол!!!! Широков - 1:0! 2-й гол в пл...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",Голкипер СКА Микко Коскинен сделал шатаут в тр...,"СКА – «Локомотив». Коскинен сыграл на ноль, от..."
3,1092758,1038079919,Кубок Гагарина,СКА,Динамо Москва,3 минута. гол!!!! Шипачев - 1:0! 5-й гол в пле...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,СКА обыграл московское «Динамо» (4:0) в четвер...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...
4,787214,1020441523,Россия. Премьер-лига,Ахмат,Волга НН,32 минута. Первое предупреждение не заставило ...,«Волга» вылетела в ФНЛ,Уступив в матче 29-го тура чемпионата России п...,«Волга» вылетела в ФНЛ & Уступив в матче 29-го...
...,...,...,...,...,...,...,...,...,...
8744,963066,1029756828,Россия. Премьер-лига,Арсенал Тула,Ахмат,23 минута. гол! 0:1! голеада Мбенгуе продолжае...,«Арсенал» Тула – «Терек» – 1:1. Команды разошл...,В 28-м туре чемпионата России «Арсенал» разоше...,«Арсенал» Тула – «Терек» – 1:1. Команды разошл...
8745,478494,141113093,Кубок Стэнли,Нью-Джерси,Лос-Анджелес,10 минута. ФРЭЗЕР - 0:1!!! 1-й гол в плей-офф!...,НХЛ. «Лос-Анджелес» одержал девятую подряд вые...,"«Лос-Анджелес», обыграв «Нью-Джерси» (2:1 ОТ) ...",НХЛ. «Лос-Анджелес» одержал девятую подряд вые...
8746,815435,158241356,КХЛ,СКА,Авангард,1 минута. СЕРГЕЙ КОСТИЦЫН - 0:1!!! 10-й гол в ...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,Сегодня в Петербурге проходит матч регулярного...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...
8747,1013451,1027691296,Кубок Гагарина,Барыс,Авангард,7 минута. гол! 1:0! Руденко! Получил шайбу в у...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,Сегодня в четвертой игре серии первого раунда ...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...


In [ ]:
columns = ['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'news_name_body']

X_train_am, X_eval_am = train_test_split(data[columns].sample(frac=1, random_state=42), test_size=1_000, random_state=42)

X_train_am, Test_am = train_test_split(X_train_am, test_size=150, random_state=42)

In [ ]:
print(len(X_train_am), len(X_eval_am), len(Test_am))

7599 1000 150


In [ ]:
X_train_am.to_csv('X_train_am.csv', index=False, encoding='utf-8')
X_eval_am.to_csv('X_eval_am.csv', index=False, encoding='utf-8')
Test_am.to_csv('Test_am.csv', index=False, encoding='utf-8')

### Обобщающие новости

#### Fine-tuning модели mbart-large-cc25

In [ ]:
SEED = 3
lang = 'ru_RU'
set_seed(SEED)

In [ ]:
dataset = load_dataset("csv", data_files={'train': 'X_train_am.csv', 'eval': 'X_eval_am.csv', 'test': 'Test_am.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e49b8615f164cede/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train'][0]

{'match_id': 965329,
 'news_id': 1029966958,
 'name': 'Германия. Бундеслига',
 'team1': 'Бавария',
 'team2': 'Майнц',
 'content': '25 минута. гол! Левандовски открывает счет! Бьет по центру, вратарь прыгает в угол! 46 минута. Первая замена у "Баварии": Вайзер вместо Мюллера. 48 минута. гол! 2:0! Швайнштайгер! Заброс Боатенга в штрафную, Левандовски катит Швайни, удар верхом - и 2:0! 51 минута. И первая замена у гостей: Окадзаки вместо Бросински. 54 минута. Роде меняет Хаби Алонсо. 72 минута. Вторая замена у "Майнца": Неделев вместо Малли. 73 минута. Левандовски уже не станет лучшим снайпером бундеслиги. Поляк уступает место Писарро. 84 минута. Последняя замена у гостей: Моритц вместо Гайса. ',
 'news_name_body': '«Бавария» – «Майнц» – 2:0. Швайнштайгер поставил точку в матче & В 34-м туре бундеслиги «Бавария» обыграла «Майнц» (2:0). Sports.ru провел текстовую онлайн-трансляцию матча . Таблица бундеслиги Статистика бундеслиги'}

In [ ]:
model_name = "facebook/mbart-large-cc25"

config = transformers.AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)

In [ ]:
model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(lang)

In [ ]:
tokenizer.src_lang = lang
tokenizer.tgt_lang = lang

# Для mbart нужен токен языка
forced_bos_token_id = (tokenizer.lang_code_to_id[lang])

model.config.forced_bos_token_id = forced_bos_token_id

In [ ]:
text_column = 'content'
summary_column = 'news_name_body'

In [ ]:
max_source_length = 512
max_target_length = 300
padding = 'max_length'
ignore_pad_token_for_loss = True

In [ ]:
def preprocess_function(examples):
  inputs, targets = [], []

  for i in range(len(examples[text_column])):
      if examples['name'][i] and examples['team1'][i] and examples['team2'][i] and examples[text_column][i] and examples[summary_column][i]:
          inputs.append('</s>'.join([examples['name'][i], examples['team1'][i], examples['team2'][i], examples[text_column][i]]))
          targets.append(examples[summary_column][i])

  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)

  # Для игнорирования паддинга в loss
  if padding == "max_length" and ignore_pad_token_for_loss:
      labels["input_ids"] = [
          [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
      ]

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [ ]:
train_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=2,
    remove_columns=['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'news_name_body'],
    )

Map (num_proc=2):   0%|          | 0/7599 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
# Data collator
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
# Метрика
metric = evaluate.load("rouge")

In [ ]:
def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]

  # rougeLSum ожидает новую строку после каждого предложения
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

In [ ]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  if ignore_pad_token_for_loss:
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  result = metric.compute(predictions=decoded_preds, references=decoded_labels, tokenizer=lambda x: x.split())
  result = {k: round(v * 100, 4) for k, v in result.items()}
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  result["gen_len"] = np.mean(prediction_lens)
  return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="adafactor",
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

train_result = trainer.train()

You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.437800,1.233596,34.995700,17.480500,29.947100,33.204700,96.800000
2,1.187200,1.127231,35.527200,18.363400,30.563300,33.800300,98.816000
2,1.044000,1.102025,36.146700,18.931800,31.190100,34.478500,97.318000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [ ]:
trainer.save_model('/content/drive/MyDrive/RuSportSum/save_model')

#### Оценка качества настроенного mBART на тестовом наборе данных

In [ ]:
predict_results = trainer.predict(
    train_dataset['test'],
    metric_key_prefix="predict",
    max_length=max_target_length,
    num_beams=3
)
metrics = predict_results.metrics

In [ ]:
metrics

{'predict_loss': 1.2225552797317505,
 'predict_rouge1': 34.0319,
 'predict_rouge2': 17.1242,
 'predict_rougeL': 29.1965,
 'predict_rougeLsum': 32.2801,
 'predict_gen_len': 95.60666666666667,
 'predict_runtime': 157.4296,
 'predict_samples_per_second': 0.953,
 'predict_steps_per_second': 0.241}

In [ ]:
predictions = []
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
    predictions = tokenizer.batch_decode(
        predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    predictions = [pred.strip() for pred in predictions]

In [ ]:
output_prediction_file = "generated_predictions.txt"
with open(output_prediction_file, "w", encoding='utf-8') as writer:
  writer.write("\n".join(predictions).encode('utf-8').decode("utf-8"))

In [ ]:
print('часть матча: ', dataset['test'][0]['content'])
print('оригинальная новость: ', dataset['test'][0]['news_name_body'])
print('предсказанная новость: ', predictions[0])

часть матча:  21 минута. гол!!! Хааааалк!!! 0:1!! Дикань не справился с мощным выстрелом бразильца - его дезориентировал небольшой рикошет от ноги защитника. И мяч от рук кипера предательски заполз за линию ворот. 27 минута. гол!!! Это Дзюба!!! 1:1!!! Со второго этажа уложил мяч прямо в угол!! Комбаров Дмитрий сделал отличный мягкий навес, а Алвеш и Янбаев вдвоем не смогли навязать внятной борьбы форварду. 32 минута. Файзулин не сможет продолжить матч. Вместо него Быстров. 41 минута. гол!!! Витцель!!! 1:2!! Шикарный дальний удар низом удался бельгийцу! Никто не встретил бельгийца, и тот воспользовался ситуацией и положил мяч в самый угол. Первый гол Акселя за "Зенит". 44 минута. Данни отловил удар в голень от Кирилла Комбарова. Прямой ногой пошел спартаковец. 64 минута. Пареха совершает прыжок двумя ногами против Денисова. Желтая карточка. 65 минута. Керж выходит на поле. Вместо Халка. 67 минута. И "Спартак" использует замену. Айрон Эйден is back!! Вместо Ананидзе. 69 минута. Пареха! У

In [ ]:
print('часть матча: ', dataset['test'][1]['content'])
print('оригинальная новость: ', dataset['test'][1]['news_name_body'])
print('предсказанная новость: ', predictions[1])

часть матча:  13 минута. Энгелланд против Сестито! Бойцы обмениваются сериями по очереди. Кажется, у обоих удары проходили. В концовке Сестито предпочел завалить оппонента на лед, ибо получил пару раз. 14 минута. ЭДЛЕР - 0:1!!! 2-й гол в чемпионате! Вратари чудят на всех континентах. Щелчок из средней зоны вратарь "Питтсбурга" выронил. В свои ворота, конечно же, куда еще. 16 минута. КРОСБИ - 1:1!!! 7-й гол в чемпионате! Очень злой был щелчок Дюпуи в контратаке, напоминал выстрел Эдлера. Типа, нате вам в обратку! А Кросби со своей излюбленной точки, стоя в метре справа у ворот, добивает. Эдлер вел себя очень пассивно - прижался к Луо, и ворота не закрывал, и Кросби не помешал. 29 минута. КУНИЦ - 2:1!!! 4-й гол в чемпионате! Блестящий розыгрыш, пас Малкина из-за ворот идеален, Куницу остается забить в пустой угол! Кросби набирает второе очко. 33 минута. РИЧАРДСОН - 2:2!!! 3-й гол в чемпионате! Красавец Уайс, нечего сказать! Устоял после силового приема на левом краю, сохранил шайбу и пер

In [ ]:
print('часть матча: ', dataset['test'][2]['content'])
print('оригинальная новость: ', dataset['test'][2]['news_name_body'])
print('предсказанная новость: ', predictions[2])

часть матча:  18 минута. А после этого Торбинский еще и ударил Даирра по ногам и схватил карточку. 29 минута. Денисову пришлось ставить корпус Виллиану после того, как тот пробросил мяч мимо него. 34 минута. гол!!! Самедов! Вот это удар! 0:1! Вывели Александра на свободное пространство, тот не стал ждать, пока его настигнут защитники, и так бабахнул, что мама не горюй! В верхний угол, да еще от штанги! 40 минута. гол!! 1:1! Эх, Чорлука. Крешич уже вышел на перехват прострела, не самого опасного, кстати, но тут вмешался Ведран и не нашел ничего лучше, чем в подкате срезать мяч в угол своих же ворот. 53 минута. Безбородов невольно помог "Анжи" отобрать мяч и начать атаку. Виллиан улетел по флангу, Тарасов сыграл ему в плечо и получил карточку. 53 минута. Еще одна карточка Чорлуке за разговоры. 69 минута. Павлюченко и Обинна выходят вместо Ндойе и Самедова. 73 минута. гол!! Это'Ооооо! 2:1! Все-таки смогли хозяева воплотить в гол угловой! Жусилей прекрасно выиграл воздух, скинул на штангу 